<a href="https://colab.research.google.com/github/chawbel/medical_chabot/blob/main/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers faiss-cpu fastapi uvicorn pyngrok python-dotenv sentence-transformers sacremoses pymed langdetect

In [ ]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import numpy as np

qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2')
translator_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
translator_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")


Device set to use cuda:0


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
from pymed import PubMed

pubmed = PubMed()
results = pubmed.query("diabetes symptoms", max_results=10)
documents = [article.abstract for article in results if article.abstract]

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.cdc.gov/diabetes/symptoms/index.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
text = soup.get_text()

In [ ]:
def preprocess(text):
  return text.replace("\n", ' ').strip()

processed_text = [preprocess(doc) for doc in documents]

In [ ]:
embedding = model.encode(processed_text)

In [ ]:
import faiss
import numpy as np

dimension = embedding.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding)

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from langdetect import detect
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    pipeline,
)
import faiss

app = FastAPI()

# Load models
# Translation: Arabic -> English
ar_to_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
ar_to_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

# Translation: English -> Arabic
en_to_ar_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
en_to_ar_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

# Embedding model (LaBSE)
embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

# QA model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

embeddings = embedding_model.encode(processed_text)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

class QueryRequest(BaseModel):
    query: str

@app.post("/answer")
async def answer(request: QueryRequest):
    try:
        # Detect language
        original_language = detect(request.query)

        # Translate query to English if needed
        if original_language == "ar":
            inputs = ar_to_en_tokenizer(request.query, return_tensors="pt")
            translated = ar_to_en_model.generate(**inputs)
            query_en = ar_to_en_tokenizer.decode(translated[0], skip_special_tokens=True)
        else:
            query_en = request.query

        # Generate query embedding
        query_embedding = embedding_model.encode([query_en])

        # Retrieve relevant document
        if index.ntotal == 0:
            raise HTTPException(status_code=500, detail="FAISS index is empty")

        distances, indices = index.search(query_embedding, k=1)
        context = processed_text[indices[0][0]]

        # Generate answer in English
        qa_result = qa_pipeline(question=query_en, context=context)
        answer_en = qa_result["answer"]

        # Translate answer back to original language if needed
        if original_language == "ar":
            inputs = en_to_ar_tokenizer(answer_en, return_tensors="pt")
            translated = en_to_ar_model.generate(**inputs)
            answer = en_to_ar_tokenizer.decode(translated[0], skip_special_tokens=True)
        else:
            answer = answer_en

        return {
            "original_query": request.query,
            "answer": answer,
            "language": original_language
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Processing failed: {str(e)}")

Device set to use cuda:0


In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import threading

# Set ngrok authentication token
ngrok.set_auth_token(userdata.get('NGROK_TOKEN'))

# Apply nest_asyncio (Colab requires this to avoid event loop issues)
nest_asyncio.apply()

# Define a function to run Uvicorn
def run_app():
    uvicorn.run(app, host='0.0.0.0', port=8000)

# Start the Uvicorn server in a separate thread
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Create an ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")


INFO:     Started server process [29435]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://e6ac-35-185-133-94.ngrok-free.app


In [ ]:
response = requests.post(
    "https://e6ac-35-185-133-94.ngrok-free.app/answer",
    json={"query": "كيفية تجنب مرض السكري"}
)
print(response.json())
# Output: {"answer": "Symptoms include increased thirst, frequent urination, and fatigue."}

INFO:     35.185.133.94:0 - "POST /answer HTTP/1.1" 200 OK
{'original_query': 'كيفية تجنب مرض السكري', 'answer': 'الحمل يقلل من حساسية الانسولين', 'language': 'ar'}
